In [1]:
from deep_nmmo.envs.team_based_env.env_configs.custom_competition_config import CustomCompetitionConfig

import nmmo
from nmmo import config
from nmmo.io import action

from neurips2022nmmo.scripted import baselines
from neurips2022nmmo import Team
from neurips2022nmmo import CompetitionConfig, scripted, RollOut, TeamBasedEnv

import ray
ray.shutdown()
ray.init()

from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog

import gym

from IPython.display import display

2022-09-01 11:54:42,851	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


ImportError: ray.tune in ray > 0.7.5 requires 'tabulate'. Please re-run 'pip install ray[tune]' or 'pip install ray[rllib]'.

In [ ]:
conf = CustomCompetitionConfig()
display_config_params = False

if display_config_params:
    print('-'*100)
    print('Displaying Config Parameters')
    print('-'*100)
    for attr in dir(conf):
        if not attr.startswith('__'):
            try:
                print(f'{attr}: {getattr(conf, attr)}')
            except:
                print(f'Unable to display attr {attr}')
    print('-'*100)

# init env
env = TeamBasedEnv